In [1]:
import os
import json

from PIL import Image

import torch
import torch.utils.data as data # для использования классов Dataset и Dataloader
import torchvision
import torchvision.transforms.v2 as tfs # для преобразования изображения в тензор

In [2]:
class MNISTDigitDataset(data.Dataset):
    def __init__(self, path, train=True, transform=None):
        self.root = path
        self.path = os.path.join(self.root, "train" if train else "test")
        self.transform = transform

        with open(os.path.join(self.root, "format.json"), "r") as fp:
            self.format = json.load(fp)

        self.length = 0 # размер обучающей выборки
        self.files = [] # список файлов изображений
        self.targets = torch.eye(10) # целевые значения

        for _dir, _target in self.format.items():
            path = os.path.join(self.path, _dir)
            list_files = os.listdir(path)
            self.length += len(list_files)
            self.files.extend(
                map(lambda _x: (os.path.join(path, _x), _target), list_files)
            )

    def __getitem__(self, item):
        # возвращает один образ из выборки
        path_file, target = self.files[item]
        t = self.targets[target] # выделяем из единичной матрицы нужную строку по значению класса
        img = Image.open(path_file)

        if self.transform:
            img = self.transform(img).ravel().float() / 255.0 # трансформируем в тензор и нормируем

        return img, t

    def __len__(self):
        return self.length

In [3]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm # визуализация процесса обучения

In [4]:
class MNISTDigitNN(nn.Module):
    def __init__(self, input_dim, num_hidden, output_dim):
        super().__init__()
        self.layer1 = nn.Linear(input_dim, num_hidden)
        self.layer2 = nn.Linear(num_hidden, output_dim)

    def forward(self, x):
        x = self.layer1(x)
        x = nn.functional.relu(x) # функция активации ReLU
        x = self.layer2(x)

        return x
    

model = MNISTDigitNN(28 * 28, 32, 10) # задаем количество нейронов на каждом слое
st = model.state_dict() # словарь состояний, хранит веса и смещения

In [5]:
to_tensor = tfs.ToImage()  # PILToTensor
# при использовании полносвязных слоев, входное изображение вытягивают в один вектор при подаче на вход сети
d_train = MNISTDigitDataset("dataset", transform=to_tensor)

print(f"Количество объектов в обучающей выборке: {len(d_train)}")

train_data = data.DataLoader(d_train, batch_size=32, shuffle=True) # batch_size - 32 файла, shuffle - перемешиваем
it = iter(train_data)
x, y = next(it) # next(it) - один батч

# print(x.size(), y.size())
# print(y)

optimizer = optim.Adam(params=model.parameters(), lr=0.01)
loss_function = nn.CrossEntropyLoss()
epochs = 2

model.train()

for _e in range(epochs):
    loss_mean = 0 # среднее значение функции потерь, выводим в консоль в процессе обучения
    lm_count = 0

    train_tqdm = tqdm(train_data, leave=True)
    for x_train, y_train in train_tqdm:
        predict = model(x_train)
        loss = loss_function(predict, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        lm_count += 1
        loss_mean = 1/lm_count * loss.item() + (1 - 1/lm_count) * loss_mean # рекрентная формула
        train_tqdm.set_description(f"Epoch [{_e+1}/{epochs}], loss_mean={loss_mean:.3f}")

Количество объектов в обучающей выборке: 60000


Epoch [2/2], loss_mean=0.194: 100%|██████████| 1875/1875 [00:21<00:00, 86.86it/s] 


In [6]:
d_test = MNISTDigitDataset("dataset", train=False, transform=to_tensor) # тестовая выборка
test_data = data.DataLoader(d_test, batch_size=500, shuffle=False) # для тестирования не имеет смысла перемешивать выборку

Q = 0

# тестирование обученной НС
model.eval()

for x_test, y_test in test_data:
    with torch.no_grad():
        p = model(x_test)
        p = torch.argmax(p, dim=1)
        y = torch.argmax(y_test, dim=1) # выбираем максимальные из 10 значений 
        Q += torch.sum(p == y).item()

Q /= len(d_test) # доля правильных ответов
print(Q)

0.9444


In [7]:
torch.save(st, 'model_MNIST.tar') # сохраняем модель (веса и смещения) как архив

In [ ]:
# загрузка модели
st2 = torch.load('model_MNIST.tar', weights_only=True, map_location="cpu") 
# weights_only=True - выполняется загрузка примитивных типов данных
# map_location="cpu" загружаем на CPU, можно выбрать map_location="cuda"

model.load_state_dict(st2) # передаем веса и смещения в модель

<All keys matched successfully>

In [ ]:
d_test = ImageFolder("dataset/test", transform=transforms) # тестовая выборка
test_data = data.DataLoader(d_test, batch_size=500, shuffle=False) # для тестирования не имеет смысла перемешивать выборку

Q = 0

# тестирование обученной НС
model.eval()

for x_test, y_test in test_data:
    with torch.no_grad():
        p = model(x_test)
        p = torch.argmax(p, dim=1) 
        Q += torch.sum(p == y_test).item()

Q /= len(d_test) # доля правильных ответов
print(Q)